In [3]:
from sklearn.impute import KNNImputer
import pandas as pd
import os
from datetime import date

x = pd.read_excel('Data/datasetFloripa.xlsx')

In [34]:
df = x
df

,NrOcorrência,Cidade,População,Data,Cidade.1,CoordenadaOcorrência,CoordenadaOBM,SubtipoOcorrência,TipoOcorrênciaAbertura,FeriadoDescrição,...,HorárioJ9,TipoOcorrênciaEncerramento,TempoTotalOcorrência,TipoOcorrênciaEncerramento.1,ClassificaçãoChamada,CidadeOBMOperacional,OBM,Bairro,lat_ocorrencia,lon_ocorrencia
0,10085306.0,Florianópolis,516524,2018-01-01 00:00:00,Florianópolis,"[-48.5835667,-27.6037201]",-,COLISÃO/CHOQUE,ATENDIMENTO PRÉ-HOSPITALAR,Confraternização Universal,...,2018-01-01 01:11:52.524000,ATENDIMENTO PRÉ-HOSPITALAR,-,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,-,COQUEIROS,-48.583567,-27.603720
1,10085307.0,Florianópolis,516524,2018-01-01 00:00:00,Florianópolis,"[-48.4997193,-27.4388923]",-,QUEDA,ATENDIMENTO PRÉ-HOSPITALAR,Confraternização Universal,...,2018-01-01 01:57:30.883000,ATENDIMENTO PRÉ-HOSPITALAR,01:38:42.929000,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,-,JURERÊ INTERNACIONAL,-48.499719,-27.438892
2,10085308.0,Florianópolis,516524,2018-01-01 00:00:00,Florianópolis,"[-48.4997041,-27.4389142]",-,-,ATENDIMENTO PRÉ-HOSPITALAR,Confraternização Universal,...,-,OCORRÊNCIA NÃO ATENDIDA,00:03:18.237000,OCORRÊNCIA NÃO ATENDIDA,-,Florianópolis,-,ESTREITO,-48.499704,-27.438914
3,10085311.0,Florianópolis,516524,2018-01-01 00:00:00,Florianópolis,"[-48.4309475,-27.5734384]",-,-,ATENDIMENTO PRÉ-HOSPITALAR,Confraternização Universal,...,2018-01-01 02:38:35.197000,ATENDIMENTO PRÉ-HOSPITALAR,01:25:10.964000,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,-,BARRA DA LAGOA,-48.430948,-27.573438
4,10085313.0,Florianópolis,516524,2018-01-01 00:00:00,Florianópolis,"[-48.5020001,-27.4465559]",-,COLISÃO/CHOQUE,ACIDENTE DE TRÂNSITO,Confraternização Universal,...,2018-01-01 05:01:24.057000,ATENDIMENTO PRÉ-HOSPITALAR,00:50:05.678000,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,-,JURERÊ INTERNACIONAL,-48.502000,-27.446556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64970,130319412.0,Florianópolis,516524,2023-07-30 00:00:00,Florianópolis,"[-48.4002807,-27.456841]","[-48.4566611974118,-27.4525590451538]",-,ATENDIMENTO PRÉ-HOSPITALAR,-,...,2023-07-30 20:28:07.303000,ATENDIMENTO PRÉ-HOSPITALAR,-,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,1º BBM - 2º/2ª/1º BBM - CANASVIEIRAS,INGLESES,-48.400281,-27.456841
64971,130319424.0,Florianópolis,516524,2023-07-30 00:00:00,Florianópolis,"[-48.6007076,-27.5957561]","[-48.5772426436301,-27.5900781786783]",-,ATENDIMENTO PRÉ-HOSPITALAR,-,...,2023-07-30 20:49:17.706000,ATENDIMENTO PRÉ-HOSPITALAR,01:09:43.896000,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,1º BBM - 1º/1ª/1º BBM - ESTREITO,CAPOEIRAS,-48.600708,-27.595756
64972,130319461.0,Florianópolis,516524,2023-07-30 00:00:00,Florianópolis,"[-48.5046855,-27.6788322]","[-48.5046556273528,-27.6696681529401]",-,ATENDIMENTO PRÉ-HOSPITALAR,-,...,2023-07-30 21:54:45.035000,OCORRÊNCIA NÃO ATENDIDA,00:28:57.666000,OCORRÊNCIA NÃO ATENDIDA,-,Florianópolis,1º BBM - 1º/1º/2ª/1º BBM - RIO TAVARES,CAMPECHE,-48.504686,-27.678832
64973,130319468.0,Florianópolis,516524,2023-07-30 00:00:00,Florianópolis,"[-48.5521704,-27.598435]","[-48.5772426436301,-27.5900781786783]",-,ATENDIMENTO PRÉ-HOSPITALAR,-,...,2023-07-30 22:20:51.180000,ATENDIMENTO PRÉ-HOSPITALAR,-,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,1º BBM - 1º/1ª/1º BBM - ESTREITO,CENTRO,-48.552170,-27.598435


In [35]:
# Convert 'NrOcorrência' column to numeric values
df['NrOcorrência'] = pd.to_numeric(x['NrOcorrência'], errors='coerce')

# Drop rows where 'NrOcorrência' is NaN
df.dropna(subset=['NrOcorrência'], inplace=True)

missing_lat = df['lat_ocorrencia'].isna().sum()
missing_lon = df['lon_ocorrencia'].isna().sum()

print("Number of missing values in 'lat_ocorrencia':", missing_lat)
print("Number of missing values in 'lon_ocorrencia':", missing_lon)

Number of missing values in 'lat_ocorrencia': 18091
Number of missing values in 'lon_ocorrencia': 18091


In [36]:
import numpy as np

# Separate 'CoordenadaOcorrência' into 'lat_ocorrencia' and 'lon_ocorrencia'
df[['lat_ocorrencia', 'lon_ocorrencia']] = df['CoordenadaOcorrência'].str.split(',', expand=True)

# Remove square brackets from the values in the new columns
df['lat_ocorrencia'] = df['lat_ocorrencia'].str.replace('[', '')
df['lon_ocorrencia'] = df['lon_ocorrencia'].str.replace(']', '')

# Remove leading and trailing spaces
df['lat_ocorrencia'] = df['lat_ocorrencia'].str.strip()
df['lon_ocorrencia'] = df['lon_ocorrencia'].str.strip()

# Replace '-' with NaN in 'lat_ocorrencia' and 'lon_ocorrencia' columns
df['lat_ocorrencia'] = df['lat_ocorrencia'].replace('-', np.nan)
df['lon_ocorrencia'] = df['lon_ocorrencia'].replace('-', np.nan)

# Convert the new columns to numeric values
df['lat_ocorrencia'] = pd.to_numeric(df['lat_ocorrencia'])
df['lon_ocorrencia'] = pd.to_numeric(df['lon_ocorrencia'])


/tmp/ipykernel_21997/2663795506.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['lat_ocorrencia'] = df['lat_ocorrencia'].str.replace('[', '')
/tmp/ipykernel_21997/2663795506.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['lon_ocorrencia'] = df['lon_ocorrencia'].str.replace(']', '')


In [37]:
df

,NrOcorrência,Cidade,População,Data,Cidade.1,CoordenadaOcorrência,CoordenadaOBM,SubtipoOcorrência,TipoOcorrênciaAbertura,FeriadoDescrição,...,HorárioJ9,TipoOcorrênciaEncerramento,TempoTotalOcorrência,TipoOcorrênciaEncerramento.1,ClassificaçãoChamada,CidadeOBMOperacional,OBM,Bairro,lat_ocorrencia,lon_ocorrencia
0,10085306.0,Florianópolis,516524,2018-01-01 00:00:00,Florianópolis,"[-48.5835667,-27.6037201]",-,COLISÃO/CHOQUE,ATENDIMENTO PRÉ-HOSPITALAR,Confraternização Universal,...,2018-01-01 01:11:52.524000,ATENDIMENTO PRÉ-HOSPITALAR,-,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,-,COQUEIROS,-48.583567,-27.603720
1,10085307.0,Florianópolis,516524,2018-01-01 00:00:00,Florianópolis,"[-48.4997193,-27.4388923]",-,QUEDA,ATENDIMENTO PRÉ-HOSPITALAR,Confraternização Universal,...,2018-01-01 01:57:30.883000,ATENDIMENTO PRÉ-HOSPITALAR,01:38:42.929000,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,-,JURERÊ INTERNACIONAL,-48.499719,-27.438892
2,10085308.0,Florianópolis,516524,2018-01-01 00:00:00,Florianópolis,"[-48.4997041,-27.4389142]",-,-,ATENDIMENTO PRÉ-HOSPITALAR,Confraternização Universal,...,-,OCORRÊNCIA NÃO ATENDIDA,00:03:18.237000,OCORRÊNCIA NÃO ATENDIDA,-,Florianópolis,-,ESTREITO,-48.499704,-27.438914
3,10085311.0,Florianópolis,516524,2018-01-01 00:00:00,Florianópolis,"[-48.4309475,-27.5734384]",-,-,ATENDIMENTO PRÉ-HOSPITALAR,Confraternização Universal,...,2018-01-01 02:38:35.197000,ATENDIMENTO PRÉ-HOSPITALAR,01:25:10.964000,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,-,BARRA DA LAGOA,-48.430948,-27.573438
4,10085313.0,Florianópolis,516524,2018-01-01 00:00:00,Florianópolis,"[-48.5020001,-27.4465559]",-,COLISÃO/CHOQUE,ACIDENTE DE TRÂNSITO,Confraternização Universal,...,2018-01-01 05:01:24.057000,ATENDIMENTO PRÉ-HOSPITALAR,00:50:05.678000,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,-,JURERÊ INTERNACIONAL,-48.502000,-27.446556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64970,130319412.0,Florianópolis,516524,2023-07-30 00:00:00,Florianópolis,"[-48.4002807,-27.456841]","[-48.4566611974118,-27.4525590451538]",-,ATENDIMENTO PRÉ-HOSPITALAR,-,...,2023-07-30 20:28:07.303000,ATENDIMENTO PRÉ-HOSPITALAR,-,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,1º BBM - 2º/2ª/1º BBM - CANASVIEIRAS,INGLESES,-48.400281,-27.456841
64971,130319424.0,Florianópolis,516524,2023-07-30 00:00:00,Florianópolis,"[-48.6007076,-27.5957561]","[-48.5772426436301,-27.5900781786783]",-,ATENDIMENTO PRÉ-HOSPITALAR,-,...,2023-07-30 20:49:17.706000,ATENDIMENTO PRÉ-HOSPITALAR,01:09:43.896000,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,1º BBM - 1º/1ª/1º BBM - ESTREITO,CAPOEIRAS,-48.600708,-27.595756
64972,130319461.0,Florianópolis,516524,2023-07-30 00:00:00,Florianópolis,"[-48.5046855,-27.6788322]","[-48.5046556273528,-27.6696681529401]",-,ATENDIMENTO PRÉ-HOSPITALAR,-,...,2023-07-30 21:54:45.035000,OCORRÊNCIA NÃO ATENDIDA,00:28:57.666000,OCORRÊNCIA NÃO ATENDIDA,-,Florianópolis,1º BBM - 1º/1º/2ª/1º BBM - RIO TAVARES,CAMPECHE,-48.504686,-27.678832
64973,130319468.0,Florianópolis,516524,2023-07-30 00:00:00,Florianópolis,"[-48.5521704,-27.598435]","[-48.5772426436301,-27.5900781786783]",-,ATENDIMENTO PRÉ-HOSPITALAR,-,...,2023-07-30 22:20:51.180000,ATENDIMENTO PRÉ-HOSPITALAR,-,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,1º BBM - 1º/1ª/1º BBM - ESTREITO,CENTRO,-48.552170,-27.598435


In [2]:
# Group the DataFrame by 'Bairro' and count the occurrences of each 'NrOcorrência'
counts_per_bairro = x.groupby('Bairro')['NrOcorrência'].count()

print(counts_per_bairro)

NameError: name 'x' is not defined

In [39]:
import pandas as pd
import numpy as np

# Step 1: Identify rows with NaN values in 'lon_ocorrencia' and 'lat_ocorrencia'
missing_values = df['lon_ocorrencia'].isnull() | df['lat_ocorrencia'].isnull()

# Step 2: Group the DataFrame by 'Bairro'
grouped = df.groupby('Bairro')

# Step 3 and 4: Impute random values based on the values in other rows of the same 'Bairro'
def impute_random(group):
    if group['lon_ocorrencia'].isnull().any() and group['lat_ocorrencia'].isnull().any():
        # Get random row indices with non-null 'lon_ocorrencia' and 'lat_ocorrencia'
        valid_indices = group[~missing_values].index

        # Check if there are valid indices to select from
        if valid_indices.empty:
            return group

        # Select a random row from the valid indices
        random_index = np.random.choice(valid_indices)

        # Get the 'lon_ocorrencia' and 'lat_ocorrencia' values from the random row
        lon_imputed = df.at[random_index, 'lon_ocorrencia']
        lat_imputed = df.at[random_index, 'lat_ocorrencia']

        # Fill NaN values with the imputed values
        group['lon_ocorrencia'].fillna(lon_imputed, inplace=True)
        group['lat_ocorrencia'].fillna(lat_imputed, inplace=True)

    return group

# Apply the imputation function to each group and concatenate the results
imputed_df = df.groupby('Bairro').apply(impute_random)

# Print the updated DataFrame
print(imputed_df)


       NrOcorrência         Cidade  População                 Data  \
0        10085306.0  Florianópolis     516524  2018-01-01 00:00:00   
1        10085307.0  Florianópolis     516524  2018-01-01 00:00:00   
2        10085308.0  Florianópolis     516524  2018-01-01 00:00:00   
3        10085311.0  Florianópolis     516524  2018-01-01 00:00:00   
4        10085313.0  Florianópolis     516524  2018-01-01 00:00:00   
...             ...            ...        ...                  ...   
64970   130319412.0  Florianópolis     516524  2023-07-30 00:00:00   
64971   130319424.0  Florianópolis     516524  2023-07-30 00:00:00   
64972   130319461.0  Florianópolis     516524  2023-07-30 00:00:00   
64973   130319468.0  Florianópolis     516524  2023-07-30 00:00:00   
64974   130319514.0  Florianópolis     516524  2023-07-31 00:00:00   

            Cidade.1       CoordenadaOcorrência  \
0      Florianópolis  [-48.5835667,-27.6037201]   
1      Florianópolis  [-48.4997193,-27.4388923]   
2     

/tmp/ipykernel_21997/2652236503.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valid_indices = group[~missing_values].index
/tmp/ipykernel_21997/2652236503.py:34: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  imputed_df = df.groupby('Bairro').apply(impute_random)


In [41]:
imputed_df

,NrOcorrência,Cidade,População,Data,Cidade.1,CoordenadaOcorrência,CoordenadaOBM,SubtipoOcorrência,TipoOcorrênciaAbertura,FeriadoDescrição,...,HorárioJ9,TipoOcorrênciaEncerramento,TempoTotalOcorrência,TipoOcorrênciaEncerramento.1,ClassificaçãoChamada,CidadeOBMOperacional,OBM,Bairro,lat_ocorrencia,lon_ocorrencia
0,10085306.0,Florianópolis,516524,2018-01-01 00:00:00,Florianópolis,"[-48.5835667,-27.6037201]",-,COLISÃO/CHOQUE,ATENDIMENTO PRÉ-HOSPITALAR,Confraternização Universal,...,2018-01-01 01:11:52.524000,ATENDIMENTO PRÉ-HOSPITALAR,-,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,-,COQUEIROS,-48.583567,-27.603720
1,10085307.0,Florianópolis,516524,2018-01-01 00:00:00,Florianópolis,"[-48.4997193,-27.4388923]",-,QUEDA,ATENDIMENTO PRÉ-HOSPITALAR,Confraternização Universal,...,2018-01-01 01:57:30.883000,ATENDIMENTO PRÉ-HOSPITALAR,01:38:42.929000,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,-,JURERÊ INTERNACIONAL,-48.499719,-27.438892
2,10085308.0,Florianópolis,516524,2018-01-01 00:00:00,Florianópolis,"[-48.4997041,-27.4389142]",-,-,ATENDIMENTO PRÉ-HOSPITALAR,Confraternização Universal,...,-,OCORRÊNCIA NÃO ATENDIDA,00:03:18.237000,OCORRÊNCIA NÃO ATENDIDA,-,Florianópolis,-,ESTREITO,-48.499704,-27.438914
3,10085311.0,Florianópolis,516524,2018-01-01 00:00:00,Florianópolis,"[-48.4309475,-27.5734384]",-,-,ATENDIMENTO PRÉ-HOSPITALAR,Confraternização Universal,...,2018-01-01 02:38:35.197000,ATENDIMENTO PRÉ-HOSPITALAR,01:25:10.964000,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,-,BARRA DA LAGOA,-48.430948,-27.573438
4,10085313.0,Florianópolis,516524,2018-01-01 00:00:00,Florianópolis,"[-48.5020001,-27.4465559]",-,COLISÃO/CHOQUE,ACIDENTE DE TRÂNSITO,Confraternização Universal,...,2018-01-01 05:01:24.057000,ATENDIMENTO PRÉ-HOSPITALAR,00:50:05.678000,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,-,JURERÊ INTERNACIONAL,-48.502000,-27.446556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64970,130319412.0,Florianópolis,516524,2023-07-30 00:00:00,Florianópolis,"[-48.4002807,-27.456841]","[-48.4566611974118,-27.4525590451538]",-,ATENDIMENTO PRÉ-HOSPITALAR,-,...,2023-07-30 20:28:07.303000,ATENDIMENTO PRÉ-HOSPITALAR,-,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,1º BBM - 2º/2ª/1º BBM - CANASVIEIRAS,INGLESES,-48.400281,-27.456841
64971,130319424.0,Florianópolis,516524,2023-07-30 00:00:00,Florianópolis,"[-48.6007076,-27.5957561]","[-48.5772426436301,-27.5900781786783]",-,ATENDIMENTO PRÉ-HOSPITALAR,-,...,2023-07-30 20:49:17.706000,ATENDIMENTO PRÉ-HOSPITALAR,01:09:43.896000,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,1º BBM - 1º/1ª/1º BBM - ESTREITO,CAPOEIRAS,-48.600708,-27.595756
64972,130319461.0,Florianópolis,516524,2023-07-30 00:00:00,Florianópolis,"[-48.5046855,-27.6788322]","[-48.5046556273528,-27.6696681529401]",-,ATENDIMENTO PRÉ-HOSPITALAR,-,...,2023-07-30 21:54:45.035000,OCORRÊNCIA NÃO ATENDIDA,00:28:57.666000,OCORRÊNCIA NÃO ATENDIDA,-,Florianópolis,1º BBM - 1º/1º/2ª/1º BBM - RIO TAVARES,CAMPECHE,-48.504686,-27.678832
64973,130319468.0,Florianópolis,516524,2023-07-30 00:00:00,Florianópolis,"[-48.5521704,-27.598435]","[-48.5772426436301,-27.5900781786783]",-,ATENDIMENTO PRÉ-HOSPITALAR,-,...,2023-07-30 22:20:51.180000,ATENDIMENTO PRÉ-HOSPITALAR,-,ATENDIMENTO PRÉ-HOSPITALAR,-,Florianópolis,1º BBM - 1º/1ª/1º BBM - ESTREITO,CENTRO,-48.552170,-27.598435


In [42]:
num_nan_lon = imputed_df['lon_ocorrencia'].isnull().sum()
num_nan_lat = imputed_df['lat_ocorrencia'].isnull().sum()

# Print the counts
print("Number of NaN values in 'lon_ocorrencia':", num_nan_lon)
print("Number of NaN values in 'lat_ocorrencia':", num_nan_lat)

Number of NaN values in 'lon_ocorrencia': 11
Number of NaN values in 'lat_ocorrencia': 11


In [44]:
num_occurrences = df['lat_ocorrencia'].value_counts().get(-48.5775211, 0)

print(f"Number of occurrences of -48.5775211 in lat_ocorrencia: {num_occurrences}")

Number of occurrences of -48.5775211 in lat_ocorrencia: 157


In [43]:
imputed_df.to_csv("Data/datasets/datasetTratadoFloripa5.csv", index=False)